# DETERMINE R function of $\theta$

- I dati nel .txt sono separati dal carattere '\t  ', disposti in 2 colonne relativi al tempo (in millisecondi) e alla funzione di correlazione

- $\sigma_{g^2}$ = 0.001 l'incertezza è fissata all'ultima ciffra significativa

- i dati validi da tenere in considerazione sono quelli nell'intervallo 0.1ms-10ms

- Gli angoli nel nome del file sono l'angolo supplementare di $\theta$ (per l'errore di misurazione)

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import os
import sys
import plotly.graph_objects as go 
dir_path = os.path.abspath('')
sys.path.append(dir_path + '/../')
from labbiofisica import Interpolazione, final_val

nel caricamento dei dati nelle strutture numpy i millisecondi vengono convertiti in secondi

In [ ]:
# Carico i dati {θ: {x: [], y: []}}

data = {}

TETA = np.array([50,60,70,80,90,100,110,120,130])
GAMMA = []
sigmaGAMMA = []

# t convertito in secondi, si considerano dati solo tra 0.1 e 10 ms a causa del rumore
for θ in TETA:
    filename = f'./data/g_{θ}.txt'

    read = pd.read_csv(filename,sep='\t ',header=None,engine='python')
    read = read[(read[0] >= 0.1) & (read[0] <= 10)] # filtro i dati

    θ = 180 - θ
    data[θ] = {'t':read[0].to_numpy()/1000,'g':read[1].to_numpy(),'sigmag': np.float64(0.001)} # converto in secondi

TETA = 180 - TETA # inverto angoli supplementari

In [ ]:
# Constants
n = 1.33 # index of refraction
η = 10**-9  # g/(nm*s) viscosity of water
λ = 633  # nm wavelength of the laser
T = 293  # K temperature
KB = 1.3806e-2  # g (nm)^2 / s^2 K boltzmann constant

interpolazione attraverso al funzione: $g^2(\tau) = A e^{-\gamma \tau} + B$

In [ ]:
# fit data
CHI_r = []
DOF = []
PVALUE = []

f = lambda x, A, B, γ: A*np.exp(-γ*x) + B

for key in data.keys():
    x = data[key]['t']
    y = data[key]['g']
    sigmay = data[key]['sigmag']
    m = Interpolazione(x, y, sigmay, f, [0.05, 0.001, 2.7],['A','B','γ'])
    data[key]['fit'] = m
    # print(m.values['γ'])
    data[key]['gamma'] = m.values['γ']
    GAMMA.append(m.values['γ'])
    sigmaGAMMA.append(m.errors['γ'])
    CHI_r.append(m.rchi2)
    DOF = m.dof
    PVALUE.append(m.pvalue)

GAMMA = np.array(GAMMA) 
sigmaGAMMA = np.array(sigmaGAMMA)


In [23]:
# print FIT RESULT:
GAMMAFVAL = [final_val(g,sg,2,3,'Hz') for g,sg in zip(GAMMA,sigmaGAMMA)]
gammateta = pd.DataFrame({'θ':TETA,'γ':GAMMA,'σγ':sigmaGAMMA,'γ_out':GAMMAFVAL,'pvalue':PVALUE,'chir':CHI_r,'dof':DOF})

print('---------------------------FIT RESULT----------------------------')
display(gammateta)

---------------------------FIT RESULT----------------------------


,θ,γ,σγ,γ_out,pvalue,chir,dof
0,130,4371.401608,181.272346,(4.37 ± 0.18)e3 Hz,1.0,0.04,50
1,120,4041.892054,155.063523,(4.04 ± 0.16)e3 Hz,1.0,0.08,50
2,110,3504.513596,127.011402,(3.5 ± 0.13)e3 Hz,1.0,0.04,50
3,100,3160.465913,102.062704,(3.16 ± 0.1)e3 Hz,1.0,0.08,50
4,90,2634.144280,76.330465,(2.63 ± 0.08)e3 Hz,1.0,0.07,50
5,80,2190.704671,58.424413,(2.19 ± 0.06)e3 Hz,1.0,0.04,50
6,70,1699.011400,44.416291,(1.7 ± 0.04)e3 Hz,1.0,0.05,50
7,60,1286.554733,32.391719,(1.29 ± 0.03)e3 Hz,1.0,0.04,50
8,50,861.021581,20.172900,(0.86 ± 0.02)e3 Hz,1.0,0.06,50


# PLOT FIT

In [24]:
colors = px.colors.sequential.Plasma

fig = go.Figure()
for i,θ in enumerate(data.keys()):
    x = data[θ]['t']
    y = data[θ]['g']
    γ = data[θ]['gamma']
    sigmay = data[θ]['sigmag']
    X,Y = data[θ]['fit'].draw()

    # plt.plot(X,Y)
    # plt.xlim(0,1)
    # plt.show()

    fig.add_trace(go.Scatter(x=X, y=Y, mode='lines',line_color=colors[i], showlegend=False,hoverinfo='skip'))

    fig.add_trace(go.Scatter(x=x,y=y, mode='markers', name=f'θ={θ}°',
        marker=dict(color = colors[i]),
        error_y=dict( 
			type='constant', 
			value=sigmay, 
			color=colors[i],
			thickness=1.5, 
			width=3, 
		)))

fig.update_layout(
        xaxis_type="log",
        yaxis_title="g<sup>2</sup>(τ)",
        xaxis_title="t (s)",
        # title='g<sup>2</sup>(τ) vs t',
        xaxis=dict(range=[np.log10(0.0001), np.log10(0.010)]),
        width=800,
        height=600,
        title={
            'text': "Correlation Function g<sup>2</sup>(τ)",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font':dict(size=30)
        },
        plot_bgcolor='white',
        font=dict(
            #family="Courier New, monospace",
            size=18,
            color="Black"
        ),
            # yaxis=dict(range=[0, 0.06])
    )

fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)

fig.show()

$Q = \frac{4\pi n}{\lambda}\sin\frac{\theta}{2}$

$\gamma = 2DQ^2 \Rightarrow D = \frac{\gamma\lambda^2}{32\pi^2n^2\sin^2\frac{\theta}{2}}$

$D = \frac{K_B T}{6\pi\eta\R}$

$R = \frac{16\pi n^2 K_B T}{3\eta\lambda^2\gamma}\sin^2\frac{\theta}{2}$

n = 1.33

$\eta$ = 0.01poise

$\lambda$ = 633nm

$K_B$ = 1.3806 $\cdot 10^{-2} g(nm)^2s^{-2}K^{-1}$

T = 293K

In [25]:
# fit gamma teta

def gamma_2DQ2(θ,D):
    theta = np.deg2rad(θ)
    return 2*D*16*(np.pi**2)*(n**2)*np.sin(theta/2)**2/(λ**2)

gammatetafit = Interpolazione(TETA, GAMMA, sigmaGAMMA, gamma_2DQ2, [0.05],['D'])
print(gammatetafit)

print('D = coefficiente diffusione = ',final_val(gammatetafit.values['D'],gammatetafit.errors['D'],2,exp=6,udm='m^2/s'))

----------------- VALORI FIT: -----------------
D: (3.705 ± 0.036)e6 

dof: 8
chi2: 14.09
chi2 ridotto: 1.76
pvalue: 0.08
------------------------------------------------

D = coefficiente diffusione =  (3.71 ± 0.04)e6 m^2/s


In [48]:
# plot fit gamma teta

fig = go.Figure()

X,Y = gammatetafit.draw()
fig.add_trace(go.Scatter(x=X, y=Y, mode='lines',line_color=px.colors.sequential.Plasma[6], showlegend=False,hoverinfo='skip'))

fig.add_trace(go.Scatter(x=TETA,y=GAMMA, mode='markers',showlegend=False,
    marker=dict(color=px.colors.sequential.Plasma[0]),
    error_y=dict( 
        type='data', 
        array=sigmaGAMMA, 
        color=px.colors.sequential.Plasma[1],
        thickness=1.5, 
        width=3, 
    )))

fig.update_layout(
    # xaxis_type="log",
    yaxis_title="γ (Hz)",
    xaxis_title="θ (°)",
    # title='g<sup>2</sup>(τ) vs t',
    # xaxis=dict(range=[np.log10(0.0001), np.log10(0.010)]),
    width=600,
    height=400,
    title={
        'text': "γ = 2DQ<sup>2</sup>(θ)",
        'y':0.8,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font':dict(size=15)
    },
    plot_bgcolor='white',
    font=dict(
        #family="Courier New, monospace",
        size=10,
        color="Black"
    ),
        # yaxis=dict(range=[0, 0.06])
)

fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
# fig.update_layout(font=dict(size=10))
fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)

fig.show()

In [49]:
def calculate_R(gamma, theta):
    theta_rad = np.radians(theta)
    R = (16 * np.pi * n**2 * T * KB * np.sin(theta_rad / 2)**2) / (3 * η * λ**2 * gamma)
    return R

def calculate_sigma_R(gamma, sigma_gamma, theta):
    theta_rad = np.radians(theta)
    sigma_R = (16 * np.pi * n**2 * T * KB * np.sin(theta_rad / 2)**2) / (3 * η * λ**2 * gamma**2) * sigma_gamma
    return sigma_R

# Calculate sigma_R for each gamma and theta
sigma_R_values = np.array([calculate_sigma_R(gamma, sigma_gamma, theta) for gamma, sigma_gamma, theta in zip(GAMMA, sigmaGAMMA, TETA)])

# Calculate R for each gamma and theta
R_values = np.array([calculate_R(gamma, theta) for gamma, theta in zip(GAMMA, TETA)])
R_out = np.array([final_val(r,sr,1,udm='nm') for r,sr in zip(R_values,sigma_R_values)])
R_values_df = pd.DataFrame({'θ (deg)': TETA, 'R (nm)': R_values,'σR (nm)': sigma_R_values,'R_out (nm)':R_out})
display(R_values_df)

,θ (deg),R (nm),σR (nm),R_out (nm)
0,130,56.222567,2.331425,56.2 ± 2.3 nm
1,120,55.520901,2.130009,55.5 ± 2.1 nm
2,110,57.290319,2.076329,57.3 ± 2.1 nm
3,100,55.556743,1.794125,55.6 ± 1.8 nm
4,90,56.795038,1.645768,56.8 ± 1.6 nm
5,80,56.432737,1.505018,56.4 ± 1.5 nm
6,70,57.938369,1.514650,57.9 ± 1.5 nm
7,60,58.142231,1.463853,58.1 ± 1.5 nm
8,50,62.067240,1.454175,62.1 ± 1.5 nm


In [50]:
R = np.average(R_values,weights=1/sigma_R_values**2)

σR = (1/np.sqrt(np.sum(1/sigma_R_values**2)))


# from statsmodels.stats.weightstats import DescrStatsW
# #calculate weighted standard deviation
# σR = DescrStatsW(R_values, weights=1/sigma_R_values**2, ddof=1).std#
# # σR = np.std(R_values,ddof=1,mean=R)

print('R = ',final_val(R,σR,1,udm='nm'))

R =  57.7 ± 0.6 nm
